In [155]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import time 

# Classical Computing

### Defining The Classical Version of Deutch Jozsa

In [175]:
def is_constant_classical(function, n):
	max_checks = 2 ** (n - 1) + 1
	first_value = function(0)
	
	for i in range(1, max_checks):
		if function(i) != first_value:
			return False  # The function is balanced
	
	return True  # The function is constant

### Defining The Classical Test Functions: One Constant & One Balanced

In [176]:
def constant_example_function(x):
	# A constant function that returns 37 for all inputs
	return 37  # This is a constant function

In [177]:
def balanced_example_function(x):
	# A balanced function that returns 0 for even x, 1 for odd x
	return x % 2  # This is a balanced function

# Quantum Computing

In [178]:
backend = Aer.get_backend('qasm_simulator')

### Implementing The Deutch Jozsa Algorithm

In [179]:
def is_constant(function, num_bits):
	qc = QuantumCircuit(num_bits + 1, num_bits)

	qc.x(num_bits)
	qc.h(range(num_bits + 1))

	qc.h(1)
	function(qc, num_bits)
	
	qc.h(range(num_bits + 1))
	
	qc.measure(range(num_bits), range(num_bits))


	transpiled_qc = transpile(qc, backend)
	job = backend.run(transpiled_qc, shots=1)
	result = job.result().get_counts()
	
	print('The oracle output after 1 call:', next(iter(result)))
	
	answer = '0' * num_bits in result
	return (answer, qc)

### Defining The Quantum Test Oracles: One Constant & One Balanced

In [180]:
def constant_example_oracle(qc, num_bits):
	# Oracle for a constant function f(x) = 0
	# This doesn't change the qubits at all
	pass
    
def balanced_example_oracle(qc, num_bits):
	# Apply CNOT gates from half of the input qubits to the output qubit
	for i in range(num_bits // 2):
		qc.cx(i, num_bits)


# Evaluation & Time Comparison

In [181]:
num_bits = 25

### Case 1: Balanced Oracle compared with Balanced Classical Function

In [182]:
start = time.time()
quantum_result_balanced, circuit = is_constant(balanced_example_oracle, num_bits)
end = time.time()
print('is constant?:', quantum_result_balanced)
print('Time taken:', end - start, '\n')
circuit.draw()

start = time.time()
classical_result_balanced = is_constant_classical(balanced_example_function, num_bits)
end = time.time()
print('is constant?:', classical_result_balanced)
print('Time taken:', end - start)


The oracle output after 1 call: 0000000000000111111111111
is constant?: False
Time taken: 0.8753776550292969 

is constant?: False
Time taken: 0.0


### Case 2: Constant Oracle compared with Constant Classical Function

In [183]:
start = time.time()
quantum_result_constant, circuit = is_constant(constant_example_oracle, num_bits)
end = time.time()
print('is constant?:', quantum_result_constant)
print('Time taken:', end - start, '\n')
circuit.draw()

start = time.time()
classical_result_constant = is_constant_classical(constant_example_function, num_bits)
end = time.time()
print('is constant?:', classical_result_constant)
print('Time taken:', end - start)

The oracle output after 1 call: 0000000000000000000000000
is constant?: True
Time taken: 0.010927677154541016 

is constant?: True
Time taken: 1.1073691844940186
